In [1]:
import pandas as pd

import requests

from bs4 import BeautifulSoup

Built the query on OECD's data explorer.
- Then select `developer API`.


url = `https://data-explorer.oecd.org/vis?lc=en&df[ds]=DisseminateArchiveDMZ&df[id]=DF_DP_LIVE&df[ag]=OECD&df[vs]=&av=true&pd=1965%2C2022&dq=AUT%2BBEL%2BCAN%2BCHL%2BCOL%2BCRI%2BCZE%2BDNK%2BEST%2BFIN%2BFRA%2BDEU%2BGRC%2BHUN%2BISL%2BIRL%2BISR%2BITA%2BJPN%2BKOR%2BLVA%2BLTU%2BLUX%2BMEX%2BNLD%2BNZL%2BNOR%2BPOL%2BPRT%2BSVK%2BSVN%2BESP%2BSWE%2BCHE%2BTUR%2BGBR%2BUSA.TAXREV..PC_GDP.A&to[TIME_PERIOD]=false&vw=tb`

In [7]:

oecd_api = 'https://sdmx.oecd.org/archive/rest/data/OECD,DF_DP_LIVE,/OAVG+AUT+BEL+CAN+CHL+COL+CRI+CZE+DNK+EST+FIN+FRA+DEU+GRC+HUN+ISL+IRL+ISR+ITA+JPN+KOR+LVA+LTU+LUX+MEX+NLD+NZL+NOR+POL+PRT+SVK+SVN+ESP+SWE+CHE+TUR+GBR+USA.TAXREV..PC_GDP.A?startPeriod=1965&endPeriod=2022'


# Make request for XML data from OECD url
response = requests.get(oecd_api)

# Parse the XML data
soup = BeautifulSoup(response.text, 'lxml')

/Users/j.i.hellings/miniconda3/envs/datasci/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [8]:
# Each observation looks like:
xml_example_obs = '''
<generic:Series>
    <generic:SeriesKey>
        <generic:Value id="LOCATION" value="GRC"/>
        <generic:Value id="INDICATOR" value="TAXREV"/>
        <generic:Value id="SUBJECT" value="TOT"/>
        <generic:Value id="MEASURE" value="PC_GDP"/>
        <generic:Value id="FREQUENCY" value="A"/>
    </generic:SeriesKey>
    <generic:Obs>
        <generic:ObsDimension id="TIME_PERIOD" value="2012"/>
        <generic:ObsValue value="36.345"/>
        <generic:Attributes>
            <generic:Value id="OBS_STATUS" value="A"/>
        </generic:Attributes>
    </generic:Obs>
</generic:Series>
'''

# Extract the data from the XML
data = []

for series in soup.find_all('generic:series'):
    location = series.find('generic:value', {'id': 'LOCATION'})['value']
    indicator = series.find('generic:value', {'id': 'INDICATOR'})['value']
    subject = series.find('generic:value', {'id': 'SUBJECT'})['value']
    measure = series.find('generic:value', {'id': 'MEASURE'})['value']
    frequency = series.find('generic:value', {'id': 'FREQUENCY'})['value']
    year = series.find('generic:obsdimension', {'id': 'TIME_PERIOD'})['value']
    value = series.find('generic:obsvalue')['value']
    data.append([location, indicator, subject, measure, frequency, year, value])

# Create a DataFrame
df = pd.DataFrame(data, columns=['location', 'indicator', 'subject', 'measure', 'frequency', 'year', 'value'])
df

,location,indicator,subject,measure,frequency,year,value
0,LTU,TAXREV,TOT,PC_GDP,A,2010,28.285
1,ISR,TAXREV,TOT,PC_GDP,A,2010,30.189
2,ESP,TAXREV,TOT,PC_GDP,A,1968,15.595
3,PRT,TAXREV,TOT,PC_GDP,A,1966,16.07
4,OAVG,TAXREV,TOT,PC_GDP,A,1967,26.228
...,...,...,...,...,...,...,...
1826,ESP,TAXREV,TOT,PC_GDP,A,2010,31.282
1827,POL,TAXREV,TOT,PC_GDP,A,2010,31.6
1828,NOR,TAXREV,TOT,PC_GDP,A,2010,41.581
1829,CRI,TAXREV,TOT,PC_GDP,A,2010,22.131


Reduce the size of the dataframe.

In [16]:
df = df[['location', 'year', 'value']].copy()
df

,location,year,value
0,LTU,2010,28.285
1,ISR,2010,30.189
2,ESP,1968,15.595
3,PRT,1966,16.07
4,OAVG,1967,26.228
...,...,...,...
1826,ESP,2010,31.282
1827,POL,2010,31.6
1828,NOR,2010,41.581
1829,CRI,2010,22.131


In [22]:
import altair as alt

import country_converter as coco

In [25]:
# Convert country codes to country names

df['country'] = coco.convert(names=df['location'], to='name_short')


# Now, our 'OAVG' location is not a country code, so we need to replace it with 'OECD - average'
df['country'] = df['country'].replace('not found', 'OECD - average')

# Replace
df

OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found in regex
OAVG not found i

,location,year,value,country
0,LTU,2010,28.285,Lithuania
1,ISR,2010,30.189,Israel
2,ESP,1968,15.595,Spain
3,PRT,1966,16.07,Portugal
4,OAVG,1967,26.228,OECD - average
...,...,...,...,...
1826,ESP,2010,31.282,Spain
1827,POL,2010,31.6,Poland
1828,NOR,2010,41.581,Norway
1829,CRI,2010,22.131,Costa Rica


In [26]:
# Let's calculate a p10 and p90 value for each year
df['value'] = df['value'].astype(float)
df['p10'] = df.groupby('year')['value'].transform(lambda x: x.quantile(0.1))
df['p90'] = df.groupby('year')['value'].transform(lambda x: x.quantile(0.9))

In [27]:
df

,location,year,value,country,p10,p90
0,LTU,2010,28.285,Lithuania,22.3039,41.8342
1,ISR,2010,30.189,Israel,22.3039,41.8342
2,ESP,1968,15.595,Spain,16.8920,34.1050
3,PRT,1966,16.070,Portugal,16.1263,32.8350
4,OAVG,1967,26.228,OECD - average,16.6538,33.3920
...,...,...,...,...,...,...
1826,ESP,2010,31.282,Spain,22.3039,41.8342
1827,POL,2010,31.600,Poland,22.3039,41.8342
1828,NOR,2010,41.581,Norway,22.3039,41.8342
1829,CRI,2010,22.131,Costa Rica,22.3039,41.8342


In [45]:
df['location'].unique()

array(['LTU', 'ISR', 'ESP', 'PRT', 'OAVG', 'USA', 'GBR', 'NOR', 'CHE',
       'NLD', 'JPN', 'FRA', 'FIN', 'ITA', 'AUT', 'LUX', 'DNK', 'CAN',
       'IRL', 'GRC', 'ISL', 'SWE', 'KOR', 'DEU', 'BEL', 'TUR', 'NZL',
       'MEX', 'POL', 'CRI', 'COL', 'HUN', 'CZE', 'LVA', 'CHL', 'SVK',
       'SVN', 'EST'], dtype=object)

In [72]:
domain = '#676A86'
styles = {
    "config": {
        "font": "Circular Std",
        "line": {"interpolate": "linear"},
        "view": {"stroke": "transparent"},
        "width": 450,
        "height": 300,
        "range": {
        "category": [
            "#179fdb",
            "#e6224b",
            "#f4c245",
            "#122b39",
            "#eb5c2e",
            "#36b7b4"
        ],
        "diverging": ["#E6224B", "#E54753", "#C9C9C9", "#179FDB", "#122B39"],
        "heatmap": ["#C9C9C9", "#179FDB", "#0063AF", "#122B39"],
        "ordinal": ["#00A767", "#36B7B4", "#179FDB", "#0063AF", "#243B5A"]
        },
        'text': {
            'color': domain,
            'align': 'left',
            'baseline': 'middle',
            'dx': 7,
            'dy': 0,
            'fontSize': 11
        },
        "axisX": {
            "labelColor": domain,
            "tickColor": domain,
            "domainColor": domain,
            "domainOpacity": 0.5,
            "gridOpacity": 0,
            "labelFont": "Circular Std",
            "labelAngle": 0,
            "labelAlign": "center",
            "labelFontSize": 11,
            "labelPadding": 5,
            "tickCount": 10,
            "tickSize": 0,
            "title": ""
        },
        "axisY": {
            "labelColor": domain,
            "labelFont": "Circular Std",
            "tickColor": domain,
            "domainColor": domain,
            "gridColor": domain,
            "gridDash": [1, 5],
            "gridOpacity": 0.5,
            "labelPadding": 5,
            "labelFontSize": 11,
            "domainOpacity": 0.5,
            "tickSize": 0,
            "title": None,
            "titleAlign": "left",
            "titleAngle": 0,
            "titleBaseline": "bottom",
            "titleColor": domain,
            "titleOpacity": 0.9,
            "titleX": 0,
            "titleY": -7
        },
        'area': {
            'opacity': 0.1,
            'color': 'dimgrey'
        }
    }
    }


In [73]:
### Let's define some configuration options for our plot.

# Apply `styles` as an Altair theme
alt.themes.register("custom_theme", lambda: styles)
alt.themes.enable("custom_theme")

ThemeRegistry.enable('custom_theme')

In [82]:
keep = ['OAVG', 'DEU', 'USA', 'GBR', 'FRA']

df_temp = df[df['location'].isin(keep)].copy()


title = alt.TitleParams(
    ['Note: Shaded area represents 10-90 percentile range', 'Source: OECD'],
    orient='bottom',
    anchor='start',
    frame='group',
    fontStyle='italic',
    fontSize=10,
    color='#676A8680',  # domain colour with 50% opacity
    fontWeight='normal',
    offset=7,
    dx=0,
)

lines = alt.Chart(df_temp, title=title).mark_line().encode(
    alt.X('year:T', timeUnit='year'),
    alt.Y('value:Q').axis(
        title='Tax revenue, percent of GDP'
    ),
    alt.Color('location:N').legend(None)
)

area = alt.Chart(df_temp).mark_area().encode(
    alt.X('year:T', timeUnit='year'),
    alt.Y('p10:Q'),
    alt.Y2('p90:Q')
)

labels = alt.Chart(df_temp).mark_text(
    dy = alt.expr("datum.location == 'GBR' ? -2 : datum.location == 'GBR' ? 1 : 0")
).encode(
    alt.X('year:T', aggregate='max'),
    alt.Y('value:Q', aggregate={'argmax':'year'}).axis(
        title='Tax revenue, percent of GDP'
    ),
    alt.Text('country:N'),
    alt.Color('location:N')
)

chart = (area + lines + labels).properties(
    width=400,
    height=280
)
chart.display()

alt.LayerChart(...)

In [86]:
chart.save('oecd_tax_burden.json')
chart.save('oecd_tax_burden.png', scale_factor=2.0)